In [1]:
import pandas as pd
from operator import itemgetter
from itertools import *

# Data Cleaning and Exploratory Data Analysis

The parsed data from the XML files is in horrible shape. We have one csv file per newpaper-page, approximately 6-10 pages per issue, 119,425 csv files in total.</br>
1. Due to the newspaper's layout headings don't necessarily precede their articles. But we can make out the headings by inspecting the single word's font sizes. In the following, we will discover how to extract the headings for one page and define a function that is applicable to all the csv files.</br> 
2. The full stops marking the end of the sentences, aren't listed as seperate words, but appended to the last word of each sentence. We have to extract the punctuation, so we can define seperate sentences.

In [2]:
# inspecting data

testpage = pd.read_csv('../data/parsed_data/1969-01-26_7.csv')
testpage

,single_word,fontsize
0,26.,size11.0
1,Januar,size11.0
2,1969,size11.0
3,/,size11.0
4,ND,size11.0
...,...,...
3633,Uhr,size9.0
3634,Ortszeit,size9.0
3635,(11.20,size9.0
3636,Uhr,size9.0


Or testpage is the 7th page of the Neues Deutschland, issued the 26th of January 1969.

#### Extracting seperate articles

In [3]:
# making out the headings by string operations and type conversion

testpage['fontsize'] = testpage['fontsize'].str.replace('size', '') # deleting the word "size" from the beginning
testpage['fontsize'] = testpage['fontsize'].astype('float16').astype('int16') # casting whole string series as integers
testpage

,single_word,fontsize
0,26.,11
1,Januar,11
2,1969,11
3,/,11
4,ND,11
...,...,...
3633,Uhr,9
3634,Ortszeit,9
3635,(11.20,9
3636,Uhr,9


We can see that there are several levels of headings ranging from font size 16 to 47, but even small font headings seem to be headings for seperate article.

In [4]:
# determining the different font sizes of the headings, assuming that all the headings have a font size larger than 11

testpage['fontsize'][testpage['fontsize']>11].value_counts()

16    33
33    28
22    12
47     6
17     5
Name: fontsize, dtype: int64

In [5]:
# inspecting the contents for the smallest headings

testpage[testpage['fontsize']==16]

,single_word,fontsize
1139,Juristenfakultät,16
1140,geschlossen,16
1220,Unruhen,16
1221,in,16
1222,Paris,16
1223,Zusammenstöße,16
1224,in,16
1225,Pakistan,16
1226,Westliche,16
1227,Journalisten,16


Headings such as "Juristenfakultät geschlossen" or "Westliche Journalisten aus der CSSR ausgewiesen" clearly stem from seperate, smaller articles, these aren't sub headings.</br>
</br>
I want to extract their corresponding articles by selecting nearby indices:

In [6]:
# reading an article
heading = ' '.join(testpage.iloc[1139:1141,0].tolist())
article = ' '.join(testpage.iloc[1141:1220,0].tolist())

print(heading)
print(article)

Juristenfakultät geschlossen
Westberlin (ADN). Die Streikbewe gung der Studenten an der Dahlemer Universität in Westberlin hält weiter an. Sie richtet sich gegen die Relegie rung von gewählten Studentenvertre tern, die sich an Solidaritätsaktionen für gemaßregelte Kommilitonen betei ligt hatten, sowie gegen die rücksichts lose Anwendung einer autoritären „Hausordnung" zur Unterdrückung der demokratischen Studentenbewegung. Als Antwort auf die Förderungen der streikenden Studenten hat der Dekan der Juristischen Fakultät, Prof. Pleyer, am Freitagabend die Schließung seiner Fakultät bis zum Ende des Semesters verkündet.


Look, what super interesting article abou the GDRs perspective on the [West German student movement](https://en.wikipedia.org/wiki/West_German_student_movement) we found!</br>
</br>
Let's come up with some code that we can use to extract all the separate articles with their headings from one page.

In [7]:
# getting all the indices according to a word's font size
heading_index = testpage[testpage['fontsize']>11].index.tolist()
article_index = testpage[testpage['fontsize']<=11].index.tolist()

# grouping both lists of indices based on the missing numbers in the sequences
heading_groups = []
for k, g in groupby(enumerate(heading_index), lambda x: x[0]-x[1]):
    heading_groups.append(list(map(itemgetter(1), g)))

article_groups = [list(map(itemgetter(1), g)) for k, g in groupby(enumerate(article_index), lambda x: x[0]-x[1])]

print(f'We found {len(heading_groups)} headings.')
print(f'We found {len(article_groups)} text segmets.')

We found 11 headings.
We found 12 text segmets.


The reason we will always find one more text segment than headings is that every page starts with some information on the date, the number of the page, a continuation of the last page's article (as in this case), or possibly something else, before the first heading. We will ignore that first text segment for now and assign each heading to the article starting below it.

In [8]:
# extracting all the headings and articles from the page and storing it in a dataframe

headings = []
articles = []
    
for i in range(len(heading_groups)):
    
    ith_heading = ' '.join(testpage.iloc[heading_groups[i][0]:heading_groups[i][-1]+1,0].tolist())
    ith_article = ' '.join(testpage.iloc[article_groups[i+1][0]:article_groups[i+1][-1]+1,0].tolist())

    headings.append(ith_heading)
    articles.append(ith_article)
    
data_df = pd.DataFrame(list(zip(headings, articles)), columns =['heading', 'article'])

data_df

,heading,article
0,Beschlüsse der Regierung der CSSR,Staatsorgane zum Ergreifen von Maßnahmen zum S...
1,CTK: Selbstmord unter Druck,Prag (ADN-Korr.). Die tschechoslowa kische Nac...
2,"Eine Million DGB- Mitglieder gegen „Vorbeugehaft""",München ' (ADN). Im Namen einer Million organi...
3,Juristenfakultät geschlossen,Westberlin (ADN). Die Streikbewe gung der Stud...
4,Unruhen in Paris Zusammenstöße in Pakistan Wes...,Stefan Jedrychowski: Bonn soll Schlußfolgerung...
5,Fall Gerstenmaier ist kerne Ausnahme,Furcht vor neuem Skandal im „braunen Bonn Eine...
6,Meinungsaustausch über Beziehungen Bulgarien -...,Neu-Delhi (ADN-Korr.). Die Minister präsidente...
7,Neue Aktionszentren für Demokratischen Fortsch...,"Die türkische Regierung beschloß ebenfalls, de..."
8,Stoltenberg übt Hinhaltetaktik Atomare Geheimv...,„Der erste Vertrag beinhaltet die Zu sammenarb...
9,Wege zum dauerhaften Frieden im Nahen Osten,"„Prawda"" zur Erfüllung der Sicherheitsratsreso..."


In [9]:
# converting our code into a function

def extracting_headings_and_articles_from_singe_page(page):
    
    heading_index = page[page['fontsize']>11].index.tolist()
    article_index = page[page['fontsize']<=11].index.tolist()

    heading_groups = []
    for k, g in groupby(enumerate(heading_index), lambda x: x[0]-x[1]):
        heading_groups.append(list(map(itemgetter(1), g)))
    article_groups = [list(map(itemgetter(1), g)) for k, g in groupby(enumerate(article_index), lambda x: x[0]-x[1])]
    
    headings = []
    articles = []
    
    for i in range(len(heading_groups)):
        
        ith_heading = ' '.join(page.iloc[heading_groups[i][0]:heading_groups[i][-1]+1,0].tolist())
        ith_article = ' '.join(page.iloc[article_groups[i+1][0]:article_groups[i+1][-1]+1,0].tolist())

        headings.append(ith_heading)
        articles.append(ith_article)
        
    data_df = pd.DataFrame(list(zip(headings, articles)), columns =['heading', 'article'])
    
    return data_df

In [10]:
# test function

extracting_headings_and_articles_from_singe_page(testpage)

,heading,article
0,Beschlüsse der Regierung der CSSR,Staatsorgane zum Ergreifen von Maßnahmen zum S...
1,CTK: Selbstmord unter Druck,Prag (ADN-Korr.). Die tschechoslowa kische Nac...
2,"Eine Million DGB- Mitglieder gegen „Vorbeugehaft""",München ' (ADN). Im Namen einer Million organi...
3,Juristenfakultät geschlossen,Westberlin (ADN). Die Streikbewe gung der Stud...
4,Unruhen in Paris Zusammenstöße in Pakistan Wes...,Stefan Jedrychowski: Bonn soll Schlußfolgerung...
5,Fall Gerstenmaier ist kerne Ausnahme,Furcht vor neuem Skandal im „braunen Bonn Eine...
6,Meinungsaustausch über Beziehungen Bulgarien -...,Neu-Delhi (ADN-Korr.). Die Minister präsidente...
7,Neue Aktionszentren für Demokratischen Fortsch...,"Die türkische Regierung beschloß ebenfalls, de..."
8,Stoltenberg übt Hinhaltetaktik Atomare Geheimv...,„Der erste Vertrag beinhaltet die Zu sammenarb...
9,Wege zum dauerhaften Frieden im Nahen Osten,"„Prawda"" zur Erfüllung der Sicherheitsratsreso..."


### Separating the Punctuation from the Words

In [11]:
# checking for the position of the full stops ('.')

testpage[testpage['single_word'].str.endswith('.')]

,single_word,fontsize
0,26.,11
14,(ADN-Korr.).,9
20,.,9
83,festgestellt.,9
142,führen.,9
...,...,...
3590,sind.,9
3592,(ADN).,9
3618,hervor.,9
3627,.gezwungen.,9
